In [21]:
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib import pyplot
from sklearn.dummy import DummyClassifier
from numpy import mean
from numpy import std
from pandas import read_csv
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [16]:
# load the dataset
def load_dataset(full_path):
	# load the dataset as a numpy array
	data = read_csv(full_path)
	# Trian Test Split
	data, test= train_test_split(data, test_size=0.1)
	# retrieve numpy array
	data = data.values
	# split into input and output elements
	X, y = data[:, :-1], data[:, -1]
	return X, y,test

In [17]:
# define the location of the dataset
full_path = 'creditcard.csv'
# load the dataset
X, y, test = load_dataset(full_path)

In [18]:
# define the reference model
model = DecisionTreeClassifier()

In [19]:
# calculate precision-recall area under curve
def pr_auc(y_true, probas_pred):
	# calculate precision-recall curve
	p, r, _ = precision_recall_curve(y_true, probas_pred)
	# calculate area under curve
	return auc(r, p)

In [20]:
# evaluate a model
def evaluate_model(X, y, model):
	# define evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# define the model evaluation the metric
	metric = make_scorer(pr_auc, needs_proba=True)
	# evaluate model
	scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
	return scores

In [7]:
# evaluate model
scores = evaluate_model(X, y, model)

In [8]:
# summarize performance
print('Mean PR AUC: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean PR AUC: 0.762 (0.042)


- Decision Tree
- k-Nearest Neighbors
- Bagged Decision Trees
- Random Forest
- Extra Trees

In [30]:
# define the reference model
DTC = DecisionTreeClassifier(criterion='gini',max_depth=2)
# evaluate model
DTCscores = evaluate_model(X, y, DTC)
# summarize performance
print('Decision Tree Classifier, gini,  max=2')
print('Mean PR AUC: %.3f (%.3f)' % (mean(DTCscores), std(DTCscores)))

Decision Tree Classifier, gini,  max=2
Mean PR AUC: 0.715 (0.107)


In [31]:
parameters = {'criterion':['gini', 'entropy']}

DTC_GS = DecisionTreeClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

metric = make_scorer(pr_auc, needs_proba=True)

clf = GridSearchCV(DTC_GS, parameters,scoring=metric,cv=cv)

clf.fit(X, y)

clf.best_params_

{'criterion': 'gini'}

In [ ]:
final_DTC_parameters = {'criterion':['gini', 'entropy'],
                        'splitter':['best', 'random'],
                        'max_depth':[None,5,10,15],
                        'min_samples_split':[2,5,10],
                        'min_samples_leaf':[1,3,5],
                        'min_weight_fraction_leaf':[0,.25,.5],
                        'max_features':[None,25,20,15,10,5,3]}

final_DTC = DecisionTreeClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

metric = make_scorer(pr_auc, needs_proba=True)

final_DTC_clf = GridSearchCV(DTC_GS, final_DTC_parameters,scoring=metric,cv=cv)

final_DTC_clf.fit(X, y)

final_DTC_clf.best_params_

In [28]:
def Grid_Search_Best_Params_(model,param_grid):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    metric = make_scorer(pr_auc, needs_proba=True)
    final = GridSearchCV(model, param_grid,scoring=metric,cv=cv)
    final.fit(X,y)
    return final.best_params_

In [ ]:
# define the reference model
DTC_opt = DecisionTreeClassifier(# will be provided above
# evaluate model
DTC_optscores = evaluate_model(X, y, model)
# summarize performance
print('Optimized Decision Tree')
print('Mean PR AUC: %.3f (%.3f)' % (mean(DTC_optscores), std(DTC_optscores)))

#### kNN

In [ ]:
final_kNN_parameters = {}

steps = [('s',StandardScaler()),('m',KNeighborsClassifier())]
final_kNN = Pipeline(steps=steps)

Grid_Search_Best_Params_(final_kNN,final_kNN_parameters)

In [ ]:
# define the reference model

In [ ]:
# define the reference model
steps = [('s',StandardScaler()),('m',KNeighborsClassifier(# will be provided above))]
kNN_opt = Pipeline(steps=steps)
# evaluate model
kNN_optscores = evaluate_model(X, y, kNN_opt)
# summarize performance
print('Optimized k Nearest Neighbors')
print('Mean PR AUC: %.3f (%.3f)' % (mean(kNN_optscores), std(kNN_optscores)))

#### BAG

In [ ]:
final_BAG_parameters = {}

final_BAG = BaggingClassifier(n_estimators=100)

best_BAG_param = Grid_Search_Best_Params_(final_BAG,final_BAG_parameters)

BAG_opt = BaggingClassifier(n_estimators=100,best_BAG_param)
# evaluate model
BAG_optscores = evaluate_model(X, y, BAG_opt)
# summarize performance
print('Optimized Bagging Classifier')
print('Mean PR AUC: %.3f (%.3f)' % (mean(BAG_optscores), std(BAG_optscores)))

### RF

In [ ]:
final_RF_parameters = {}

final_RF = RandomForestClassifier(n_estimators=100)

best_RF_param = Grid_Search_Best_Params_(final_RF,final_RF_parameters)

RF_opt = RandomForestClassifier(n_estimators=100,best_RF_param)
# evaluate model
RF_optscores = evaluate_model(X, y, RF_opt)
# summarize performance
print('Optimized Random Forest')
print('Mean PR AUC: %.3f (%.3f)' % (mean(RF_optscores), std(RF_optscores)))

#### ET

In [ ]:
final_ET_parameters = {}

final_ET = ExtraTreesClassifier(n_estimators=100)

best_ET_param = Grid_Search_Best_Params_(final_ET,final_ET_parameters)

ET_opt = RandomForestClassifier(n_estimators=100,best_ET_param)
# evaluate model
EF_optscores = evaluate_model(X, y, ET_opt)
# summarize performance
print('Optimized Extra Trees')
print('Mean PR AUC: %.3f (%.3f)' % (mean(ET_optscores), std(ET_optscores)))